In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import time
import csv
import queue
from datetime import datetime

/Users/beni/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model_path = "/Users/beni/Desktop/Climbing Technique Tracker/Climbing_Tracker/pose_landmarker_full.task"

In [3]:
"""The code in this block is partly taken from the official MediaPipe documentation and can be found 
here: https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python#live-stream_1"""

BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

saved_results = queue.Queue()


def print_result(result: PoseLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    print('pose landmarker result: {}'.format(result))
    saved_results.put(result)
    saved_results.put(timestamp_ms)


options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM, #sets the running mode to live stream
    result_callback=print_result)


"""The following code is written by myself using the OpenCV and MediaPipe documentation for help which
can be found in the readme file"""

#check how many frames per second Qsense sensors output
frames_per_second = 10
time_between_frames = 1 / frames_per_second


with PoseLandmarker.create_from_options(options) as landmarker:
    #Use openCV VideoCapture to start capturing from the webcam
    start_time = time.time()

    capture = cv2.VideoCapture(1)

    #Create a loop to read the latest frame from the camera using VideoCaptureRead()
    while capture.isOpened():

        ret, frame = capture.read()
     
        if not ret:
            print("broken")
            break

        #Convert the frame receieved from OpenCV to a MediaPipe Image Object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
      
        #landmarker.detect_async(mp_image, timestamp_ms = int(time.time() * 1000))
        #from datetime import datetime
        landmarker.detect_async(mp_image, int(datetime.now().timestamp() * 1000))


        #color = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('frame', frame) #first param is window name
        cv2.waitKey(10)

        """The following frame limiter was taken from the following online answer:
         https://www.quora.com/How-do-I-decrease-the-frames-per-second-in-OpenCV-python"""
        
        elapsed_time = time.time() - start_time 
        time_to_wait = time_between_frames - elapsed_time 
        if time_to_wait > 0: 
            time.sleep(time_to_wait)
    

I0000 00:00:1751864553.284183 9885927 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M2 Max
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751864553.360777 9886354 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751864553.371890 9886356 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


pose landmarker result: PoseLandmarkerResult(pose_landmarks=[], pose_world_landmarks=[], segmentation_masks=None)
pose landmarker result: PoseLandmarkerResult(pose_landmarks=[[NormalizedLandmark(x=0.5709754824638367, y=0.6240463852882385, z=-1.2170110940933228, visibility=0.9982037544250488, presence=0.9805759787559509), NormalizedLandmark(x=0.6009680032730103, y=0.5464858412742615, z=-1.1499606370925903, visibility=0.9962673783302307, presence=0.9505017995834351), NormalizedLandmark(x=0.618563711643219, y=0.5499851107597351, z=-1.1503121852874756, visibility=0.9973865151405334, presence=0.9440394639968872), NormalizedLandmark(x=0.6364635229110718, y=0.5532287955284119, z=-1.1504658460617065, visibility=0.994927167892456, presence=0.9328194856643677), NormalizedLandmark(x=0.5421971082687378, y=0.5397233963012695, z=-1.1655858755111694, visibility=0.9971074461936951, presence=0.962438702583313), NormalizedLandmark(x=0.521346390247345, y=0.5389289259910583, z=-1.1655433177947998, visibil

W0000 00:00:1751864554.827683 9886354 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


pose landmarker result: PoseLandmarkerResult(pose_landmarks=[[NormalizedLandmark(x=0.5734345316886902, y=0.6369964480400085, z=-1.268410086631775, visibility=0.9983871579170227, presence=0.9884462356567383), NormalizedLandmark(x=0.6003280878067017, y=0.5530442595481873, z=-1.1910489797592163, visibility=0.996854841709137, presence=0.9794928431510925), NormalizedLandmark(x=0.6178791522979736, y=0.5573131442070007, z=-1.1914432048797607, visibility=0.997795581817627, presence=0.9774751663208008), NormalizedLandmark(x=0.6354900002479553, y=0.5623942613601685, z=-1.1914774179458618, visibility=0.9957586526870728, presence=0.9726595282554626), NormalizedLandmark(x=0.542602002620697, y=0.5499756336212158, z=-1.2115685939788818, visibility=0.9974967241287231, presence=0.9810765981674194), NormalizedLandmark(x=0.5217611789703369, y=0.5523338913917542, z=-1.2116743326187134, visibility=0.9985023736953735, presence=0.979800283908844), NormalizedLandmark(x=0.5022793412208557, y=0.5552442073822021

KeyboardInterrupt: 

In [ ]:
results = []
while not saved_results.empty():
    
    landmark = saved_results.get() #get pose_landmark for each frame
    try:
        landmarks = [] #initialize an array for each indidual landmark in a frame (33 total)
        for i in range(0,32):
            landmark_point = landmark.pose_landmarks[0][i] #get each landmark (0, 32) for the 0th person in the frame
            landmarks.append(landmark_point)
    except:
        print("no landmarks detected") #first couple frames might not detect a person yet


    time_stamp = saved_results.get() #get timestamp (comes after each landmark in the queue)
    
    results.append([time_stamp]+ landmarks) #append landmarks + timestamp into results array


print("Collected", len(results), "results")


"""CSV code adapted from the following website: https://www.geeksforgeeks.org/python/writing-csv-files-in-python/"""

#each frame is writen into the CSV, each column are the coordinates for a single landmark
with open('results.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)

no landmarks detected
NormalizedLandmark(x=0.5709754824638367, y=0.6240463852882385, z=-1.2170110940933228, visibility=0.9982037544250488, presence=0.9805759787559509)
NormalizedLandmark(x=0.6009680032730103, y=0.5464858412742615, z=-1.1499606370925903, visibility=0.9962673783302307, presence=0.9505017995834351)
NormalizedLandmark(x=0.618563711643219, y=0.5499851107597351, z=-1.1503121852874756, visibility=0.9973865151405334, presence=0.9440394639968872)
NormalizedLandmark(x=0.6364635229110718, y=0.5532287955284119, z=-1.1504658460617065, visibility=0.994927167892456, presence=0.9328194856643677)
NormalizedLandmark(x=0.5421971082687378, y=0.5397233963012695, z=-1.1655858755111694, visibility=0.9971074461936951, presence=0.962438702583313)
NormalizedLandmark(x=0.521346390247345, y=0.5389289259910583, z=-1.1655433177947998, visibility=0.9982702732086182, presence=0.9626495242118835)
NormalizedLandmark(x=0.5010495185852051, y=0.539467453956604, z=-1.1661981344223022, visibility=0.99755704

In [12]:
#calculating joint angle

timestamp = 10
array = [1,2,3,4,5]

combined = []
combined.append([timestamp] + array)

print(combined)


[[10, 1, 2, 3, 4, 5]]
